In [1]:
import json
import pipeline_utils
import os

In [ ]:
root_dir = os.getcwd()
save_dir = "/mnt/home/hlethi/ceph/learningGeometry_20240627" # <- change this to your saving directory

In [ ]:

# Specify parameter inputs
# See `config.json` to see the supported input parameters
# TODO: Add comments for each input parameters
parameter_inputs = {
    "model_training": {
        "model": ["VGG11"], 
        "task": ["CIFAR-10"], # the dataset to train the model
        "optim": ["SGD"],
        "lr": [1.0], # the learning rate
        "criterion": ["mse"], # the loss function
        "num_epoch": [101], # measure statistics at epoch 0 and train 100 epochs. Set to 10001 if `use_step=True`
        "save_folder": [save_dir],
        "precision": ["double"], # whether to use "float" (float-32) or "double" (float-64). For lazy training, need to use `double` precision
        "alpha": [1.0, 1.5, 2.0, 3.0, 5.0, 7.5, 10.0, 15.0, 30.0, 50.0, 75.0, 100.0], # the scaling factor `alpha`
        "gain": [1.0],
        "decay": [0.0], # weight decay. See `torch.optim.SGD`
        "useStep": [False], # whether `num_epoch` would stand for number of epochs or number of gradient steps
        "seed": [7, 13, 17, 37, 47], # random seed to initialize the model's weight
        },
    "feature_analysis": {
        "layers": [["view"]], # names of the layers to extract features from
        "sampleFunc": ["random"], # name of the sample function to sample from the dataset
        "numSmpl": [50], # number of samples for each class
        "numCls": [10], # number of classes
        "fsData": ["CIFAR-10"], # the dataset to extract features
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98], # random seed for sampling
        "lastEpoch": [False], # whether to analyze model checkpoints of all epoch or only the last epoch
        "saveFeature": [False], # whether to save the extracted features
        "runAnalysis": [True], # whether to run the analysis
    },
    "linear_probe": {
        "fsData": ["CIFAR-100"],
        "lastEpoch": [True],
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "layer": ["view"],
        "runCapacity": [True], # whether to run capacity analysis
        "runLinearProbe": [True], # whether to run linear probe analysis
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98],
    },
    "eval_corrupt": {
        "fsData": [["gaussian_noise"], ["defocus_blur"], ["frost"], ["brightness"], ["pixelate"], ["shot_noise"], ["zoom_blur"], ["jpeg_compression"], ["impulse_noise"], ["glass_blur"], ["motion_blur"], ["snow"], ["fog"], ["contrast"], ["elastic_transform"]],
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "layer": ["view"],
        "runCapacity": [True],
        "runEvalCorrupt": [True], # whether to run the evaluation on corrupted dataset
        "corruptLevels": [[1, 2, 3, 4, 5]], # corrupt levels of the dataset. See CIFAR-10C description in https://github.com/hendrycks/robustness
        "lastEpoch": [True],
        "subSeed": [1, 4, 18, 27, 29, 49, 64, 82, 90, 98],
    },
    "kernels": {
        "sampleFunc": ["random"],
        "numSmpl": [50],
        "numCls": [10],
        "fsData": ["CIFAR-10"],
        "lastEpoch": [False],
    }
}

In [4]:
# Generate parameter list
parameter_list = pipeline_utils.generate_parameter_pipeline(parameter_inputs, "standard")
for step in parameter_list:
    print(f"Step {step}: Generated a list of {len(parameter_list[step])} parameters!")

Step model_training: Generated a list of 60 parameters!
Step feature_analysis: Generated a list of 600 parameters!


In [5]:
# Save parameter list into json file
filename = "params/params_test.json"
json.dump(parameter_list, open(filename, "w"))
print(f"Save parameter list to json file at {filename}")

Save parameter list to json file at params/params_test.json


In [6]:
n_thread = 4
for step in parameter_list:
    num_jobs = len(parameter_list[step])
    job_array_length = (num_jobs // n_thread) - 1 if num_jobs % n_thread == 0 else num_jobs // n_thread
    print(f"Step {step}: Job array length: 0-{job_array_length}")

Step model_training: Job array length: 0-14
Step feature_analysis: Job array length: 0-149
